# About Web Services

---

---

Web service idea is to access information via an *URL*, instead of a *web page* displayed in a web browser, and to get raw data.

The principle consist to build an URL with some parameters depending on the data we want to retrieve.

Parameters are generally defined by the web service specifications.

Here is describe the basics on how to use the web services developped at EMSC with Python and / or Bash shell:


[[1]](#sec_01) Practical overview

[[2]](#sec_02) First use in Python

[[3]](#sec_03) First use in Bash shell

[[4]](#sec_04) Choice of the output format

   * [[4.1]](#sec_04.1) Parsing JSON output
   * [[4.2]](#sec_04.2) Parsing QuakeML

<a id='sec_01'></a>
## 1. Practical overview

From an user perspective, a web service is just an url!
> e.g. http://www.seismicportal.eu/fdsnws/event/1/query?limit=10&start=2017-09-01&end=2017-11-01&format=text&minmag=6.5

It's composed of: 
 * a protocal: *http*,
 * an hostname: *www.seismicportal.eu*,
 * a path: */fdsnws/event/1/query* and, 
 * a query: *?limit=10&start=2017-09-01&end=2017-11-01&format=text&minmag=6.5* (...with different attributs *limit*, *start*, *format* and *minmag* and their values).

From a practical point of view, we just have to 
 1. Build the url
 2. Download the result
 3. Parse the output

<a id='sec_02'></a>
## 2. First use in Python

Let's start with the *fdsn-event* web service (http://www.seismicportal.eu/fdsn-wsevent.html).

Build a query to retrieve seismic events from September 1st to November 1st, 2017 with a magnitude greater than 6.5:

 * Web service **prefix** is: http://www.seismicportal.eu/fdsnws/event/1/query
 * **Parameters** are: *start=2017-09-01*, *end=2017-11-01*, *minmag=6.5* and *format=text*

The resulting url will be (*http://prefix\query*):
> http://www.seismicportal.eu/fdsnws/event/1/query\?start=2017-09-01&end=2017-11-01&format=text&minmag=6.5

In [209]:
# -------------------------------------
#      Main functions used below
# -------------------------------------
# Func-Download
def geturl(url):
    import requests
    from StringIO import StringIO
    res = requests.get(url, timeout=15)
    return {'status': res.status_code,
            'content': res.text}

# Func-Parse (txt)
def parsecsv(txt, usedict=False):
    import csv
    if usedict:
        parser = csv.DictReader(StringIO(txt), delimiter='|')
    else:
        parser = csv.reader(StringIO(txt), delimiter='|')
        header = parser.next()
    return [ line for line in parser]

# Func-Parse (json)
def parsejson(txt):
    import json
    return json.loads(txt)

# Func-Parse (xml)
def parsexml(url):
   from obspy import read_events
   return read_events(url)

In [210]:
#Build the url
url = "http://www.seismicportal.eu/fdsnws/event/1/query\
?start=2017-09-01&end=2017-11-01&format=text&minmag={minmag}".format(minmag=6.5)

The **geturl** Python function was previously defined in order to download data from an url:

Let's download data !

In [211]:
res = geturl(url)
print parsecsv(res['content']) # Print the download result:

[['20171031_0000002', '2017-10-31T00:42:12.4Z', '-21.71', '169.11', '25.0', 'EMSC', 'EMSC-RTS', 'EMSC', '627167', 'mw', '6.8', 'EMSC', 'SOUTHEAST OF LOYALTY ISLANDS'], ['20171024_0000031', '2017-10-24T10:47:47.2Z', '-7.33', '123.03', '548.0', 'EMSC', 'EMSC-RTS', 'EMSC', '626022', 'mw', '6.7', 'EMSC', 'BANDA SEA'], ['20171010_0000169', '2017-10-10T18:53:33.6Z', '-54.33', '8.49', '10.0', 'EMSC', 'EMSC-RTS', 'EMSC', '623367', 'mw', '6.7', 'EMSC', 'BOUVET ISLAND REGION'], ['20171008_0000103', '2017-10-08T22:34:33.5Z', '52.37', '176.83', '121.0', 'EMSC', 'EMSC-RTS', 'EMSC', '622928', 'mw', '6.6', 'EMSC', 'RAT ISLANDS, ALEUTIAN ISLANDS'], ['20170919_0000091', '2017-09-19T18:14:38.5Z', '18.59', '-98.47', '50.0', 'EMSC', 'EMSC-RTS', 'EMSC', '619258', 'mw', '7.1', 'EMSC', 'PUEBLA, MEXICO'], ['20170908_0000020', '2017-09-08T04:49:21.2Z', '15.02', '-93.81', '72.0', 'EMSC', 'EMSC-RTS', 'EMSC', '616600', 'mw', '8.1', 'EMSC', 'OFFSHORE CHIAPAS, MEXICO']]


Now we need to parse the output !

In [212]:
#Parse the result
for row in parsecsv(res['content']):
    print row

print("\nAnd we get {0} events !".format(len(row)))

['20171031_0000002', '2017-10-31T00:42:12.4Z', '-21.71', '169.11', '25.0', 'EMSC', 'EMSC-RTS', 'EMSC', '627167', 'mw', '6.8', 'EMSC', 'SOUTHEAST OF LOYALTY ISLANDS']
['20171024_0000031', '2017-10-24T10:47:47.2Z', '-7.33', '123.03', '548.0', 'EMSC', 'EMSC-RTS', 'EMSC', '626022', 'mw', '6.7', 'EMSC', 'BANDA SEA']
['20171010_0000169', '2017-10-10T18:53:33.6Z', '-54.33', '8.49', '10.0', 'EMSC', 'EMSC-RTS', 'EMSC', '623367', 'mw', '6.7', 'EMSC', 'BOUVET ISLAND REGION']
['20171008_0000103', '2017-10-08T22:34:33.5Z', '52.37', '176.83', '121.0', 'EMSC', 'EMSC-RTS', 'EMSC', '622928', 'mw', '6.6', 'EMSC', 'RAT ISLANDS, ALEUTIAN ISLANDS']
['20170919_0000091', '2017-09-19T18:14:38.5Z', '18.59', '-98.47', '50.0', 'EMSC', 'EMSC-RTS', 'EMSC', '619258', 'mw', '7.1', 'EMSC', 'PUEBLA, MEXICO']
['20170908_0000020', '2017-09-08T04:49:21.2Z', '15.02', '-93.81', '72.0', 'EMSC', 'EMSC-RTS', 'EMSC', '616600', 'mw', '8.1', 'EMSC', 'OFFSHORE CHIAPAS, MEXICO']

And we get 13 events !


<a id='sec_03'></a>
## 3. First use in Bash shell

Same example in shell. 

In [213]:
%%bash 
exec 2> /dev/null

echo "*** Url building"
MINMAG=6.5
url="http://www.seismicportal.eu/fdsnws/event/1/query?start=2017-09-01&end=2017-11-01&format=text&minmag=$MINMAG"
echo "url=${url}"

echo
echo "*** Web service output ***"
wget -O res.txt ${url}
cat res.txt

echo
echo "*** Parsing ***"
awk -F'|' '!/^#/ {printf "unid:%s, Origin Time:%s, Magnitude:%s, Region:%s\n", $1, $2, $11, $13}' res.txt

*** Url building
url=http://www.seismicportal.eu/fdsnws/event/1/query?start=2017-09-01&end=2017-11-01&format=text&minmag=6.5

*** Web service output ***
#EventID | Time | Latitude | Longitude | Depth/km | Author | Catalog | Contributor | ContributorID | MagType | Magnitude | MagAuthor | EventLocationName
20171031_0000002|2017-10-31T00:42:12.4Z|-21.71|169.11|25.0|EMSC|EMSC-RTS|EMSC|627167|mw|6.8|EMSC|SOUTHEAST OF LOYALTY ISLANDS
20171024_0000031|2017-10-24T10:47:47.2Z|-7.33|123.03|548.0|EMSC|EMSC-RTS|EMSC|626022|mw|6.7|EMSC|BANDA SEA
20171010_0000169|2017-10-10T18:53:33.6Z|-54.33|8.49|10.0|EMSC|EMSC-RTS|EMSC|623367|mw|6.7|EMSC|BOUVET ISLAND REGION
20171008_0000103|2017-10-08T22:34:33.5Z|52.37|176.83|121.0|EMSC|EMSC-RTS|EMSC|622928|mw|6.6|EMSC|RAT ISLANDS, ALEUTIAN ISLANDS
20170919_0000091|2017-09-19T18:14:38.5Z|18.59|-98.47|50.0|EMSC|EMSC-RTS|EMSC|619258|mw|7.1|EMSC|PUEBLA, MEXICO
20170908_0000020|2017-09-08T04:49:21.2Z|15.02|-93.81|72.0|EMSC|EMSC-RTS|EMSC|616600|mw|8.1|EMSC|OFFSHORE CH

<a id='sec_04'></a>
## 4. Choice of the output format

It's important to know the parameters available at the web service specification to customize the request for your exact need. For instance, the choice of the output format is important. Because it may be more or less easy to parse the results and because it depends on the information that you need.

At the moment we have chosen

> &format=text

**text** is a good choice for shell use or for human eyes.

However, for fdsn-event web service, you also have the choice of 
**json** and **xml**.



<a id='sec_04.1'></a>
### 4.1 Parsing JSON output

In [214]:
url = "http://www.seismicportal.eu/fdsnws/event/1/query?start=2017-09-01&end=2017-11-01&minmag=6.5&format=json"

Note the **&format=json** in the url. Now we get the content:

In [215]:
info = geturl(url)
print info['content'][:400]

{"type":"FeatureCollection","metadata":{"totalCount":6},"features":[{
  "geometry": {
    "type": "Point", 
    "coordinates": [
      169.11, 
      -21.71, 
      -25.0
    ]
  }, 
  "type": "Feature", 
  "id": "20171031_0000002", 
  "properties": {
    "lastupdate": "2017-10-31T06:24:00.0Z", 
    "magtype": "mw", 
    "evtype": "ke", 
    "lon": 169.11, 
    "auth": "EMSC", 
    "lat": -21.71, 


And finally the parsing process:

In [216]:
data = parsejson(info['content'])

print "Type of the parsed data:", type(data)
print "List of keys:", data.keys()

Type of the parsed data: <type 'dict'>
List of keys: [u'type', u'features', u'metadata']


In [217]:
# Event informations are in the 'features' attribute:
for ev in data['features']:
    print("==> Event: {0}\n{1}\n".format(ev['id'], ev['properties']))

==> Event: 20171031_0000002
{u'lastupdate': u'2017-10-31T06:24:00.0Z', u'magtype': u'mw', u'time': u'2017-10-31T00:42:12.4Z', u'lon': 169.11, u'auth': u'EMSC', u'source_id': u'627167', u'depth': 25.0, u'unid': u'20171031_0000002', u'mag': 6.8, u'evtype': u'ke', u'lat': -21.71, u'source_catalog': u'EMSC-RTS', u'flynn_region': u'SOUTHEAST OF LOYALTY ISLANDS'}

==> Event: 20171024_0000031
{u'lastupdate': u'2017-10-24T11:06:00.0Z', u'magtype': u'mw', u'time': u'2017-10-24T10:47:47.2Z', u'lon': 123.03, u'auth': u'EMSC', u'source_id': u'626022', u'depth': 548.0, u'unid': u'20171024_0000031', u'mag': 6.7, u'evtype': u'ke', u'lat': -7.33, u'source_catalog': u'EMSC-RTS', u'flynn_region': u'BANDA SEA'}

==> Event: 20171010_0000169
{u'lastupdate': u'2017-10-11T06:50:00.0Z', u'magtype': u'mw', u'time': u'2017-10-10T18:53:33.6Z', u'lon': 8.49, u'auth': u'EMSC', u'source_id': u'623367', u'depth': 10.0, u'unid': u'20171010_0000169', u'mag': 6.7, u'evtype': u'ke', u'lat': -54.33, u'source_catalog': u'

<a id='sec_04.2'></a>
### 4.2 Parsing QuakeML

For QuakeML, the easiest choice is to use the parser from Obspy.

In [218]:
url = "http://www.seismicportal.eu/fdsnws/event/1/query?start=2017-09-01&end=2017-11-01&minmag=6.5&format=xml"
data = parsexml(url)
print data

6 Event(s) in Catalog:
2017-10-31T00:42:12.400000Z | -21.710, +169.110 | 6.8 mw | manual
2017-10-24T10:47:47.200000Z |  -7.330, +123.030 | 6.7 mw | automatic
2017-10-10T18:53:33.600000Z | -54.330,   +8.490 | 6.7 mw | manual
2017-10-08T22:34:33.500000Z | +52.370, +176.830 | 6.6 mw | manual
2017-09-19T18:14:38.500000Z | +18.590,  -98.470 | 7.1 mw | manual
2017-09-08T04:49:21.200000Z | +15.020,  -93.810 | 8.1 mw | manual
